In [1]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-30 15:13:52--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-30 15:13:52 (31.6 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [3]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [4]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [5]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [6]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [7]:
llm('write that this is a test')

' This statement, "This is a test," is typically used to indicate the initial testing phase of software or any system. It serves multiple purposes:\n\n1. **Identification**: Early in development, it\'s crucial for developers and quality assurance (QA) engineers to identify potential issues, bugs, or areas that require improvement within a system. By running tests during this phase, the team can ensure smooth progression into subsequent stages of development.\n\n2. **Functionality Check**: The statement helps in verifying whether fundamental functionalities are working as intended before more complex features and systems are implemented. This could range from simple unit tests to checks for integration among components.\n\n3. **Documentation and Communication**: It\'s a standardized phrase used globally, making it easy for anyone involved in the project, regardless of their role, to recognize that a test is being conducted. This helps in maintaining clear communication across teams and 

In [8]:
print(_)

 This statement, "This is a test," is typically used to indicate the initial testing phase of software or any system. It serves multiple purposes:

1. **Identification**: Early in development, it's crucial for developers and quality assurance (QA) engineers to identify potential issues, bugs, or areas that require improvement within a system. By running tests during this phase, the team can ensure smooth progression into subsequent stages of development.

2. **Functionality Check**: The statement helps in verifying whether fundamental functionalities are working as intended before more complex features and systems are implemented. This could range from simple unit tests to checks for integration among components.

3. **Documentation and Communication**: It's a standardized phrase used globally, making it easy for anyone involved in the project, regardless of their role, to recognize that a test is being conducted. This helps in maintaining clear communication across teams and documenta